# Tutorial: webscraping en LinkedIn con python

Introduccion mamalon

Este tutorial se realizo en un equipo de computo con Ubuntu 16.04 y con Python 3.5

# Background

<div style="width:image width px; font-size:80%; text-align:center;"><img src="img/interes_tiempo.png" alt="alternate text" width="width" height="height" style="padding-bottom:0.5em;" />Interes a lo largo del tiempo para el temino <strong>web scraping</strong></div>

La principal razon por la que se eligio hacer este tutorial con python es por su popularidad, y por su facilidad de implementacion

<div style="width:image width px; font-size:80%; text-align:center;"><img src="img/consultas_relacionadas.png" alt="alternate text" width="width" height="height" style="padding-bottom:0.5em;" />Consultas relacionadas a <strong>web scraping</strong></div>

# Manos a la obra!

## Ambientacion

### Instalar selenium

Para instalar selenium, bastara con ejecutar el comando: <br/>
<code>pip install -U selenium</code> </br>

Selenium es un entorno de pruebas de software para aplicaciones basadas en la web. En otras palabras es posible automatizar la realizacion de ciertas acciones en aplicaciones web, nosotros usaremos esta habilidad para simular hacer busquedas en la plataforma.

Selenium requiere drivers para interacturar como un navegador. Firefox por ejemplo requiere geckodriver, que necesita ser instalado antes de poder correr con selenium. Asegurate de agregarlo a tu PATH, por ejemplo para sistemas linux en /usr/bin o /usr/local/bin o agregalo al ~/.bashrc

Te adjuntamos las ligas en la tabla de abajo

<table>
<colgroup>
<col>
<col>
</colgroup>
<tbody>
<tr><td><strong>Chrome</strong>:</td>
<td><a href="https://sites.google.com/a/chromium.org/chromedriver/downloads" rel="nofollow">https://sites.google.com/a/chromium.org/chromedriver/downloads</a></td>
</tr>
<tr><td><strong>Edge</strong>:</td>
<td><a href="https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/" rel="nofollow">https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/</a></td>
</tr>
<tr><td><strong>Firefox</strong>:</td>
<td><a href="https://github.com/mozilla/geckodriver/releases" rel="nofollow">https://github.com/mozilla/geckodriver/releases</a></td>
</tr>
<tr><td><strong>Safari</strong>:</td>
<td><a href="https://webkit.org/blog/6900/webdriver-support-in-safari-10/" rel="nofollow">https://webkit.org/blog/6900/webdriver-support-in-safari-10/</a></td>
</tr>
</tbody>
</table>

## Proceso

### Paso 1: Importar librerias

En caso de marcar error en alguna libreria, basta con ejecutar 

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import pickle
from functools import reduce

### Paso 2: Generar Cookies con el Login

Una vez instaladas las librerias y los drivers, necesitamos una sesion para poder navegar libremente por la plataforma, para eso es necesario abrir la sesion con el driver y posteriormente guardar los cookies generados con la libreria:
<div style="width:image width px; font-size:80%; text-align:center;"><img src="img/login_linked.gif" alt="alternate text" width="width" height="height" style="padding-bottom:0.5em;" />Ejemplo del proceso a seguir para guardar cookie</div>

### Paso 3: Cargar cookie y realizar una busqueda en LinkendIn

In [107]:
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains
# Funcion para cargar cookie guardada
def load_cookie(driver, path):
     with open(path, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            driver.add_cookie(cookie)
# Este codigo lo obtuve revisando la liga que genera linkendIn cada vez que filtramos por ciudad
places = {"cdmx":"%5B%22mx%3A5921%22%5D"}
# Palabras que queremos buscar
keywords = ["ceo", "developer"]

# Genera la ruta que abriremos con el driver
# La funcion usa la teconologia de busqueda implementada por linkedIn
#  https://www.linkedin.com/help/linkedin/answer/76723/usa-la-busqueda-booleana-en-linkedin?lang=es
def _search_people_link(place, keywords, num_page):
    keywords_string = reduce(lambda a,b: "{} O {}".format(a, b), keywords)
    keywords_string = keywords_string.replace(" ","%20")
    return "https://www.linkedin.com/search/results/people/?facetGeoRegion={}&keywords={}&origin=FACETED_SEARCH&page={}".format(place, keywords_string, num_page)
def get_contacts(driver):
    content = BeautifulSoup(driver.page_source, 'lxml')
    contacts = content.findAll("div", {"class": "search-result__info"})
    array_contacts = []
    for contact in contacts:
        con={}
        con["name"] = contact.findAll("span", {"class": "name actor-name"} )[0].contents[0]
        con["position"] = contact.findAll("span", {"dir": "ltr"})[0].contents[0]
        con["location"] = contact.findAll("span", {"dir": "ltr"})[1].contents[0]
        array_contacts.append(con)
    return array_contacts

# Creando sesion con Firefox
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.get(login_path)
# Recuperando sesion con las cookies
load_cookie(driver, cookie_name)
# Esperando a que la plataforme cargue la sesion
driver.implicitly_wait(500)

contacts = []
past_num = 0
for page in range(1,1000):
    # Generando url con busqueda
    url = _search_people_link(places["cdmx"], keywords, page)
    driver.get(url)
    driver.implicitly_wait(100)
    # Javascript renderiza solo lo que el usuario puede ver
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight*(1/ 5));")
    driver.implicitly_wait(100)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight*(2/ 5));")
    driver.implicitly_wait(100)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight*(3/ 5));")
    driver.implicitly_wait(100)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.implicitly_wait(100)
    contacts = contacts + get_contacts(driver)
    driver.implicitly_wait(100)
    if(past_num == len(contacts)):
        break;
    past_num=len(contacts)
    

In [108]:
len(contacts)

114

<div style="width:image width px; font-size:80%; text-align:center;"><img src="img/automated_linkedIn.gif" alt="alternate text" width="width" height="height" style="padding-bottom:0.5em;" />Busqueda automatizada</div>

# Paso 3: obteniendo informacion del documento

A este punto podremos acceder a toda la informacion disponible en la pagina que hemos cargado,
lo siguiente es tratar la informacion:

In [83]:
content = BeautifulSoup(driver.page_source, 'lxml')

In [84]:
contacts = content.findAll("div", {"class": "search-result__info"})

In [85]:
len(contacts)

10

[<span dir="ltr">Chief Executive Officer en Nomad Republic</span>,
 <span dir="ltr">Ciudad de México y alrededores, México</span>]

In [90]:
contacts[0].findAll("div", {"class": "search-result__info"})

<div class="search-result__info pt3 pb4 ph0">
<a class="search-result__result-link ember-view" data-control-id="yDhF4SsZSHql9F+Fh0j9zg==" data-control-name="search_srp_result" href="/in/santiago-espinosa-de-los-monteros-harispuru-01ba9011b/" id="ember63"> <h3 class="actor-name-with-distance search-result__title single-line-truncate ember-view" id="ember64"><span class="name-and-icon"><span class="name-and-distance">
<span class="name actor-name">Santiago Espinosa de los Monteros Harispuru</span>
<span class="distance-badge separator ember-view" id="ember65"> <span class="visually-hidden">contacto de 2º grado</span>
<span class="dist-value">2º</span>
</span>
</span><!-- --></span>
</h3>
</a> <p class="subline-level-1 t-14 t-black t-normal search-result__truncate">
<span dir="ltr">Chief Executive Officer en Nomad Republic</span>
</p>
<p class="subline-level-2 t-12 t-black--light t-normal search-result__truncate">
<span dir="ltr">Ciudad de México y alrededores, México</span>
</p>
<p class

In [99]:
array_contacts = []
for contact in contacts:
    con={}
    con["name"] = contact.findAll("span", {"class": "name actor-name"} )[0].contents[0]
    con["position"] = contact.findAll("span", {"dir": "ltr"})[0].contents[0]
    con["location"] = contact.findAll("span", {"dir": "ltr"})[1].contents[0]
    array_contacts.append(con)

In [ ]:
array_contacts

In [77]:
content.get_attribute_list("")

[None]

In [26]:
a = contacts[0]

In [34]:
c = a.findAll("span", {"class": "name actor-name"} )

In [37]:
d = c[0]

In [41]:
d.contents[]

['Santiago Espinosa de los Monteros Harispuru']

In [32]:
a.findAll

<div class="search-result__info pt3 pb4 ph0">
<a class="search-result__result-link ember-view" data-control-id="ZKfZ0MFpSPKGU8vQjvQtJQ==" data-control-name="search_srp_result" href="/in/santiago-espinosa-de-los-monteros-harispuru-01ba9011b/" id="ember63"> <h3 class="actor-name-with-distance search-result__title single-line-truncate ember-view" id="ember64"><span class="name-and-icon"><span class="name-and-distance">
<span class="name actor-name">Santiago Espinosa de los Monteros Harispuru</span>
<span class="distance-badge separator ember-view" id="ember65"> <span class="visually-hidden">contacto de 2º grado</span>
<span class="dist-value">2º</span>
</span>
</span><!-- --></span>
</h3>
</a> <p class="subline-level-1 t-14 t-black t-normal search-result__truncate">
<span dir="ltr">Chief Executive Officer en Nomad Republic</span>
</p>
<p class="subline-level-2 t-12 t-black--light t-normal search-result__truncate">
<span dir="ltr">Ciudad de México y alrededores, México</span>
</p>
<p class

## Resultados

## Conclusion

## Fuentes

## Instalar selenium

## El driver

La libreria que usaremos para simular la entrada a la pagina es <strong>selenium</strong>, para eso necesitaremos el driver de algun navegador instalado, en este caso el driver que usaremos es: ; el driver nos servira para ingresar a la plataforma como si lo hicieramos desde un navegador firefox

# Fuentes
https://pypi.org/project/selenium/